<a href="https://colab.research.google.com/github/hr7git/Data-Analysis-with-Open-Source/blob/main/14%EA%B0%95%EB%B3%B4%EC%B6%A9_LLM_%EC%8B%A4%EC%8A%B5_level_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

14강의 기존 코드 내용이 초보자에게는 어려워서 내용을 좀더 단순화 해서 새로운 코드를 만들었습니다.  

이번 코드를 먼저 생각해보거서 실제 14강을 해보면 훨씬 내용이 수월하게 접근이 될 것입니다.

기존의 복잡한 VLM(이미지 분석) 과정을 빼고, **간단한 텍스트 데이터 분석 후 EXAONE 모델이 보고서를 작성**하는 구조로 최적화했습니다.

---

### 🛠️ 수정된 프로그램 코드 ( 실습용)

이 코드의 필수 라이브러리 설치 부분은 14강을 참조해서 설치하고 아래의 부분은 예시이지만 오류가 난다는 것에 주의 할것.
* 특히 pipeline 은 소규모LLM 을 위한 것이라서 LG-exaone 같은 LLM 을 감당 할 수 없어서 vlm 을 이용해야 함.  ( 아래의 코드에서 주의 할 것! )

```python
# 1. 필수 라이브러리 설치
# !pip install "numpy<2.0"
# !pip install -q transformers datasets evaluate accelerate vllm

# import pandas as pd
# from transformers import pipeline
# from vllm import LLM, SamplingParams

# --- 1단계: 맛집 리뷰 데이터 준비 (Pandas) ---
data = {
    'menu': ['치즈 피자', '매운 파스타', '마라 떡볶이', '갈릭 브레드', '아이스 아메리카노'],
    'review': [
        "치즈가 엄청 늘어나고 맛있어요! 그런데 좀 짜요.",
        "너무 매워서 먹기가 힘들었어요. 소스가 너무 강해요.",
        "인생 떡볶이입니다. 마라향이 적절해서 좋았어요.",
        "빵이 너무 딱딱해서 턱이 아팠어요. 실망입니다.",
        "산미가 적당하고 시원해서 피자랑 잘 어울려요!"
    ]
}
df = pd.DataFrame(data)

# --- 2단계: 감성 분석 수행 (Hugging Face Pipeline) ---
# 가벼운 모델을 사용하여 리뷰가 긍정인지 부정인지 분류합니다.
classifier = pipeline("sentiment-analysis", model="snunlp/KR-FinBert-SC")

print("리뷰 분석 중...")
df['analysis'] = df['review'].apply(lambda x: classifier(x)[0])
df['label'] = df['analysis'].apply(lambda x: x['label'])
df['score'] = df['analysis'].apply(lambda x: x['score'])

# --- 3단계: EXAONE 모델 로드 (vLLM 활용) ---
# 기존 14강의 EXAONE 로드 방식을 유지하되, 메모리 효율을 위해 vLLM을 사용합니다.
llm = LLM(model="LGAI-Research/EXAONE-3.0-7.8B-Instruct", tensor_parallel_size=1)

# --- 4단계: 프롬프트 구성 (14강 스타일) ---
prompt = [
    {"role": "system", "content": "당신은 친절하고 전문적인 식당 경영 컨설턴트입니다."}
]

# 데이터프레임의 내용을 하나씩 프롬프트에 추가 (Chaining)
for row in df.itertuples():
    status = "👍긍정적" if row.label == "positive" else "👎개선필요"
    prompt.append({
        "role": "user",
        "content": f"메뉴: {row.menu}\n리뷰: {row.review}\nAI분석: {status} (신뢰도: {row.score:.2f})"
    })

# 마지막 보고서 작성 요청
prompt.append({
    "role": "user",
    "content": "위의 데이터들을 종합하여 사장님께 드리는 '주간 주방 운영 개선 리포트'를 전문적인 마크다운 형식으로 작성해주세요."
})

# --- 5단계: 최종 보고서 생성 및 출력 ---
sampling_params = SamplingParams(temperature=0.3, top_p=0.95, max_tokens=2048)
outputs = llm.chat(prompt, sampling_params)

print("\n" + "="*50)
print("📝 EXAONE이 생성한 컨설팅 보고서")
print("="*50 + "\n")
print(outputs[0].outputs[0].text)

```

---

### 💡 주요 수정 사항 설명

1. **데이터 소스 변경**: 복잡한 RSS 이미지 크롤링 대신, 직접 정의한 **맛집 리뷰 DataFrame**을 사용합니다. (이해가 훨씬 빠릅니다!)
2. **VLM 제거 및 NLP 도입**: 사진을 분석하던 `MiniCPM-V` 대신, 텍스트 감정을 읽는 `pipeline("sentiment-analysis")`를 사용해 분석 단계를 단순화했습니다.
3. **14강 로직 유지**:  `df.itertuples()`를 이용해 데이터를 하나씩 `prompt.append` 하는 **프롬프트 체이닝 기법**은 그대로 유지했습니다. 이 방식이 나중에 대량의 회계 데이터를 처리할 때도 핵심이 됩니다.
4. **EXAONE 활용**: 마지막 보고서 작성은 여전히 강력한 한국어 모델인 **EXAONE**이 담당하게 하여 결과물의 퀄리티를 높였습니다.



# 기본 환경 만들기
* install version 을 down grade
* 한글처리
* 필수 라이브러리 설치

In [ ]:
!pip install "numpy<2.0"

In [ ]:

# 4bit VLM 처리를 위한 bitsandbytes 설치
# LLM 처리를 위한 VLLM 설치 (오래걸리는 작업(>5분)이므로 미리 실행!)
!pip install bitsandbytes==0.45.3 vllm==0.7.3 transformers==4.48.2
# 필요 시 세션 재시작

In [ ]:
# 한글 처리를 위한 matplotlib 설정 (1)

!sudo apt-get install -y fonts-nanum
!sudo fc-cache –fv
!rm ~/.cache/matplotlib -rf

Session restart

In [ ]:
# 한글 처리를 위한 matplotlib 설정 (2)

import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')

In [ ]:
# 1. 필수 라이브러리 설치

import pandas as pd
import numpy as np
from transformers import pipeline
from vllm import LLM, SamplingParams


# 코드 작업 단계별 실행

In [ ]:
# --- 1단계: 맛집 리뷰 데이터 준비 (Pandas) ---
data = {
    'menu': ['치즈 피자', '매운 파스타', '마라 떡볶이', '갈릭 브레드', '아이스 아메리카노'],
    'review': [
        "치즈가 엄청 늘어나고 맛있어요! 그런데 좀 짜요.",
        "너무 매워서 먹기가 힘들었어요. 소스가 너무 강해요.",
        "인생 떡볶이입니다. 마라향이 적절해서 좋았어요.",
        "빵이 너무 딱딱해서 턱이 아팠어요. 실망입니다.",
        "산미가 적당하고 시원해서 피자랑 잘 어울려요!"
    ]
}
df = pd.DataFrame(data)

In [ ]:
# --- 2단계: 감성 분석 수행 (Hugging Face Pipeline) ---
# 가벼운 모델을 사용하여 리뷰가 긍정인지 부정인지 분류합니다.
classifier = pipeline("sentiment-analysis", model="snunlp/KR-FinBert-SC")

print("리뷰 분석 중...")
df['analysis'] = df['review'].apply(lambda x: classifier(x)[0])
df['label'] = df['analysis'].apply(lambda x: x['label'])
df['score'] = df['analysis'].apply(lambda x: x['score'])


In [ ]:
# --- 3단계: EXAONE 모델 로드 (vLLM 활용) ---
# 기존 14강의 EXAONE 로드 방식을 유지하되, 메모리 효율을 위해 vLLM을 사용합니다.
# llm = LLM(model="LGAI-Research/EXAONE-3.0-7.8B-Instruct", tensor_parallel_size=1)



# from vllm import LLM, SamplingParams

## vLLM 라이브러리를 사용하여 'LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct' 모델을 로드
## gpu_memory_utilization은 GPU 메모리 사용 비율을 0.5로 설정
## max_model_len은 모델이 처리할 수 있는 최대 토큰 길이를 10000으로 설정
llm = LLM(model='LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct',gpu_memory_utilization=0.5, max_model_len = 1000)

In [ ]:
# --- 4단계: 프롬프트 구성 (14강 스타일) ---
prompt = [
    {"role": "system", "content": "당신은 친절하고 전문적인 식당 경영 컨설턴트입니다."}
]

# 데이터프레임의 내용을 하나씩 프롬프트에 추가 (Chaining)
for row in df.itertuples():
    status = "👍긍정적" if row.label == "positive" else "👎개선필요"
    prompt.append({
        "role": "user",
        "content": f"메뉴: {row.menu}\n리뷰: {row.review}\nAI분석: {status} (신뢰도: {row.score:.2f})"
    })

# 마지막 보고서 작성 요청
prompt.append({
    "role": "user",
    "content": "위의 데이터들을 종합하여 사장님께 드리는 '주간 주방 운영 개선 리포트'를 전문적인 마크다운 형식으로 작성해주세요."
})


In [ ]:
# --- 5단계: 최종 보고서 생성 및 출력 ---
sampling_params = SamplingParams(temperature=0.3, top_p=0.95, max_tokens=2048)
outputs = llm.chat(prompt, sampling_params)

print("\n" + "="*50)
print("📝 EXAONE이 생성한 컨설팅 보고서")
print("="*50 + "\n")
print(outputs[0].outputs[0].text)

INFO 02-01 06:40:15 chat_utils.py:332] Detected the chat template content format to be 'string'. You can set `--chat-template-content-format` to override this.


Processed prompts: 100%|██████████| 1/1 [00:16<00:00, 16.25s/it, est. speed input: 17.42 toks/s, output: 39.34 toks/s]


📝 EXAONE이 생성한 컨설팅 보고서

# 주간 주방 운영 개선 리포트

## 개요
본 리포트는 최근 주간 동안 제공된 다양한 메뉴에 대한 고객 리뷰를 분석하여 주방 운영 및 메뉴 개발 측면에서의 개선 사항을 제시합니다. 주요 메뉴와 그에 따른 피드백을 바탕으로 효율적인 서비스 향상을 목표로 합니다.

## 메뉴별 분석 및 개선 사항

### 1. **치즈 피자**
- **리뷰 요약**: 치즈의 품질과 맛은 뛰어나지만, 짠맛이 강해 고객 만족도가 다소 떨어짐.
- **개선 사항**:
  - **맛 균형 조정**: 짠맛 조절을 위한 소스 조절 또는 다양한 맛의 치즈 조합 고려.
  - **추가 옵션**: 저염 옵션 제공을 고려하여 다양한 입맛을 만족시키기.

### 2. **매운 파스타**
- **리뷰 요약**: 매운 정도가 너무 강해 고객 경험에 부정적 영향을 미침.
- **개선 사항**:
  - **맵기 조절**: 고객 선호도에 맞춘 맵기 조절 옵션 제공 (예: 중간 맵기, 덜 매운 옵션).
  - **소스 균형**: 매운 맛과 함께 다른 맛 요소를 균형 있게 추가하여 복합적인 맛 제공.

### 3. **마라 떡볶이**
- **리뷰 요약**: 마라향이 적절하게 조화를 이루어 호평 받으나, 일부 고객은 여전히 맵기 조절이 필요함.
- **개선 사항**:
  - **맵기 조절 옵션**: 고객의 맵기 선호도에 따라 다양한 맵기 수준을 제공하는 옵션 추가.
  - **다양성 강화**: 다른 향신료나 재료를 조합하여 다양한 맛의 마라 떡볶이 개발 고려.

### 4. **갈릭 브레드**
- **리뷰 요약**: 빵의 딱딱함으로 인해 고객 불편함이 보고됨.
- **개선 사항**:
  - **빵 품질 개선**: 반죽 및 굽기 과정 최적화를 통해 부드러움 향상.
  - **고객 피드백 반영**: 정기적인 고객 피드백 수집 및 반영 시스템 구축.

### 5. **아이스 아메리카노**
- **리뷰 요약**: 산미와 시원함이 장점이나, 피자와의 조화에 대한 언급이 있어 메